In [56]:
!pip install pytesseract
!pip install opencv-python
!sudo apt-get install tesseract-ocr -y
!pip install pytesseract opencv-python-headless

import cv2
import pytesseract






Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [79]:
video_path = "/content/drive/MyDrive/202_video/b2_p3_4.mp4"

In [93]:
import cv2
import pytesseract
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import os

# 设置 Tesseract OCR 的路径
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

def convert_to_24hr_format(timestamp):
    """将时间戳转换为24小时制格式"""
    try:
        # 假设时间戳是类似 "12:34:56 PM" 或 "01:23:45 AM" 格式
        dt = datetime.strptime(timestamp, "%I:%M:%S")  # 12小时制格式
        hr = dt.hour
        # print(hr)
        if hr < 8:
            # print(hr)
            hr += 12
            dt = dt.replace(hour=hr)
        # else:/ = dt./replace(hour=hr - 12)
        return dt.strftime("%H:%M:%S")  # 转换为24小时制
        # els/turn dt.strftime("%H:%M:%S")
    except ValueError:
        return timestamp  # 如果转换失败，直接返回原始时间戳

def timestamp_difference_is_greater_than_one_second(prev_timestamp, curr_timestamp):
    """检查两个时间戳之间的差异是否大于 1 秒"""
    try:
        # 将时间戳转换为 datetime 对象
        prev_time = datetime.strptime(prev_timestamp, "%H:%M:%S")
        curr_time = datetime.strptime(curr_timestamp, "%H:%M:%S")

        # 计算时间差，单位为秒
        time_diff = (curr_time - prev_time).total_seconds()

        # 如果差值大于 1 秒，打印错误信息
        if time_diff > 1:
            print(f"Error: Timestamp difference exceeds 1 second. Prev: {prev_timestamp}, Curr: {curr_timestamp}")
            return True  # 返回 True 表示有问题
        return False  # 差值不超过 1 秒
    except ValueError:
        return False  # 如果时间戳格式不正确，则返回 False

def extract_timestamp_from_video(video_path, crop_size=(50, 430), frame_output_interval=15):
    """从视频中提取时间戳"""
    # 打开视频文件
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    timestamps = []  # 保存提取的时间戳
    prev_timestamp = None  # 上一个时间戳

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # 视频结束

        # 裁剪左上角区域
        top_left_crop = frame[0:crop_size[0], 230:crop_size[1]]

        # 转为灰度图像
        gray = cv2.cvtColor(top_left_crop, cv2.COLOR_BGR2GRAY)

        # 二值化处理
        _, thresh = cv2.threshold(gray, 241, 255, cv2.THRESH_BINARY)

        # 可选：反转颜色，如果背景是黑色，文字是白色
        thresh = cv2.bitwise_not(thresh)

        # OCR 提取时间戳
        custom_config = r'--oem 3 --psm 7 -c tessedit_char_whitelist=0123456789:.'  # 只允许数字和冒号
        timestamp = pytesseract.image_to_string(thresh, config=custom_config).strip()

        if timestamp:  # 如果成功提取到内容
            timestamp_24hr = convert_to_24hr_format(timestamp)  # 转换为24小时制格式

            # 判断时间戳与上一个时间戳的差异是否大于1秒
            if prev_timestamp and timestamp_difference_is_greater_than_one_second(prev_timestamp, timestamp_24hr):
                # 如果时间戳差值大于1秒，打印错误信息
                plt.figure(figsize=(10, 6))

                # 原始裁剪区域
                plt.subplot(1, 2, 1)
                plt.imshow(cv2.cvtColor(top_left_crop, cv2.COLOR_BGR2RGB))
                plt.title("Cropped Region")
                plt.axis('off')

                # 二值化后的图像
                plt.subplot(1, 2, 2)
                plt.imshow(thresh, cmap='gray')
                plt.title("Binary Threshold")
                plt.axis('off')

                # 显示图片
                plt.show()
                pass  # 错误信息已经在 `timestamp_difference_is_greater_than_one_second` 函数中打印
            else:
                timestamps.append((frame_count, timestamp_24hr))

            # 每隔一定数量的帧输出一次
            if frame_count % frame_output_interval == 0:
                print(f"Frame {frame_count}: {timestamp_24hr}")

            # 更新上一个时间戳
            prev_timestamp = timestamp_24hr
        else:
            # 如果没有提取到时间戳，输出错误信息
            print(f"Error: No timestamp extracted at frame {frame_count}")

        frame_count += 1

    cap.release()
    return timestamps

def process_videos_in_folder(folder_path):
    """遍历文件夹中的所有视频文件并处理"""
    # 获取文件夹内的所有文件
    files = os.listdir(folder_path)

    # 遍历文件夹中的每个文件
    for file in files:
        file_path = os.path.join(folder_path, file)

        # 检查文件是否是视频文件（以 .mp4 或其他视频格式结尾）
        if file.endswith(('.mp4', '.avi', '.mov', '.mkv')):
            print(f"Processing video: {file}")
            timestamps = extract_timestamp_from_video(file_path)

            # 打印所有提取的时间戳
            # print(f"Extracted Timestamps for {file}:")
            # for frame, timestamp in timestamps:
            #     print(f"Frame {frame}: {timestamp}")
        else:
            print(f"Skipping non-video file: {file}")

# 运行函数，指定文件夹路径
folder_path = "/content/drive/MyDrive/202_video"  # 替换为视频文件夹的路径
process_videos_in_folder(folder_path)

Processing video: b1_p1_1.mp4
Frame 0: 13:25:48
Frame 15: 13:25:49
Frame 30: 13:25:50
Frame 45: 13:25:51


KeyboardInterrupt: 